In [1]:
from ultralytics import YOLO

model = YOLO('../models/best.pt')

In [2]:
def tojson(self, normalize=False):
    """Convert the object to JSON format."""
    if self.probs is not None:
        LOGGER.warning('Warning: Classify task do not support `tojson` yet.')
        return

    import json

    # Create list of detection dictionaries
    results = []
    data = self.boxes.data.cpu().tolist()
    h, w = self.orig_shape if normalize else (1, 1)
    for i, row in enumerate(data):  # xyxy, track_id if tracking, conf, class_id
        box = {'x1': row[0] / w, 'y1': row[1] / h, 'x2': row[2] / w, 'y2': row[3] / h}
        conf = row[-2]
        class_id = int(row[-1])
        name = self.names[class_id]
        result = {'name': name, 'class': class_id, 'confidence': conf, 'box': box}
        if self.boxes.is_track:
            result['track_id'] = int(row[-3])  # track ID
        if self.masks:
            x, y = self.masks.xy[i][:, 0], self.masks.xy[i][:, 1]  # numpy array
            result['segments'] = {'x': (x / w).tolist(), 'y': (y / h).tolist()}
        if self.keypoints is not None:
            x, y, visible = self.keypoints[i].data[0].cpu().unbind(dim=1)  # torch Tensor
            result['keypoints'] = {'x': (x / w).tolist(), 'y': (y / h).tolist(), 'visible': visible.tolist()}
        results.append(result)

    # Convert detections to JSON
    return json.dumps(results, indent=2)

In [4]:
results = model('../for_predict/pan_cake.png')


image 1/1 c:\Users\User\Desktop\CRUSH ON AI \\code\..\for_predict\pan_cake.png: 640x640 1 pan_cake, 2 strawberrys, 325.1ms
Speed: 3.0ms preprocess, 325.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
import json

to_json = tojson(results[0])
json_object = json.loads(to_json)
json_object

[{'name': 'pan_cake',
  'class': 109,
  'confidence': 0.9206794500350952,
  'box': {'x1': 171.84237670898438,
   'y1': 212.4482421875,
   'x2': 790.22705078125,
   'y2': 771.7550048828125}},
 {'name': 'strawberry',
  'class': 137,
  'confidence': 0.742880642414093,
  'box': {'x1': 140.48587036132812,
   'y1': 715.2039184570312,
   'x2': 313.2257385253906,
   'y2': 888.1173706054688}},
 {'name': 'strawberry',
  'class': 137,
  'confidence': 0.7033928036689758,
  'box': {'x1': 301.91839599609375,
   'y1': 738.7453002929688,
   'x2': 447.1363830566406,
   'y2': 872.853515625}}]

In [6]:
class_list = []

for i in range(len(json_object)):
    name = json_object[i]['name']
    if name in class_list:
        pass
    else:
        class_list.append(name)

class_list

['pan_cake', 'strawberry']